# Ejercicio:
Dada la data de geyser.csv, utilizar el agloritmo Gaussiano para entrenar en la detección de anomalías. Intentar probar con diferentes percentiles.

El conjunto de datos contiene información sobre las erupciones del géiser Old Faithful localizado en el parque nacional de Yellowstone, Wyoming. En concreto, recoge información sobre la duración de 299 erupciones, así como el tiempo transcurrido desde la anterior

## Lectura conjunto de datos y EDA

In [3]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import matplotlib.gridspec as gridspec
from collections import Counter
from sklearn import metrics
import numpy as np
from matplotlib.colors import LogNorm
from sklearn.metrics import f1_score
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import StandardScaler

/var/folders/79/rwrch1qx1y7bm5mc3hnnntw00000gn/T/ipykernel_77410/478755733.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [4]:
df_geyser = pd.read_csv('geyser.csv')
df_geyser

,duration,waiting,kind
0,3.600,79,long
1,1.800,54,short
2,3.333,74,long
3,2.283,62,short
4,4.533,85,long
...,...,...,...
267,4.117,81,long
268,2.150,46,short
269,4.417,90,long
270,1.817,46,short


In [5]:
# Comprobamos si alguna columna tiene valores nulos
df_geyser.isna().any()

duration    False
waiting     False
kind        False
dtype: bool

## Detección de anomalías con Distribución Gaussiana

In [6]:
# Se reduce el DataFrame eliminando la columna 'kind'
df_geyser.drop(columns='kind', inplace=True)
df_geyser.head()

,duration,waiting
0,3.600,79
1,1.800,54
2,3.333,74
3,2.283,62
4,4.533,85


In [7]:
# Escalado de los datos:
scaler = StandardScaler()
df_geyser_scaled = scaler.fit_transform(df_geyser)
df_geyser_scaled

array([[ 0.09849886,  0.59712344],
       [-1.48145856, -1.24518118],
       [-0.13586149,  0.22866251],
       [-1.05750332, -0.6556437 ],
       [ 0.91744345,  1.03927655],
       [-0.53085085, -1.171489  ],
       [ 1.06402839,  1.2603531 ],
       [ 0.09849886,  1.03927655],
       [-1.34979544, -1.46625773],
       [ 0.75681445,  1.03927655],
       [-1.45249268, -1.24518118],
       [ 0.37674691,  0.96558436],
       [ 0.62515133,  0.52343125],
       [-1.52534627, -1.76102647],
       [ 1.06402839,  0.89189218],
       [-1.1593228 , -1.39256555],
       [-1.52534627, -0.6556437 ],
       [ 1.1518038 ,  0.96558436],
       [-1.65700939, -1.39256555],
       [ 0.66903904,  0.59712344],
       [-1.48145856, -1.46625773],
       [-1.52534627, -1.76102647],
       [-0.03316426,  0.52343125],
       [-0.36934409, -0.13979841],
       [ 0.91744345,  0.22866251],
       [ 0.09849886,  0.89189218],
       [-1.33487362, -1.171489  ],
       [ 0.5224541 ,  0.37604688],
       [ 0.31793739,

### Entrenamiento del algoritmo

In [8]:
# Se entrena el algoritmo Gaussiano
gm = GaussianMixture(n_components=2, random_state=42)
gm.fit(df_geyser_scaled)

GaussianMixture(n_components=2, random_state=42)

### Anomalías identificadas

Para la identificación de las anomalías, se debe seleccionar un threshold a partir del cual, todos los ejemplos que se encuentren en regiones con una densidad menor a la indicada en el threshold, se consideran anomalías.

#### Percentile 0.03

In [13]:
# Selección del Threshold
densities = gm.score_samples(df_geyser_scaled)
density_threshold = np.percentile(densities, 0.03)
print("Threshold seleccionado:", density_threshold)

Threshold seleccionado: -6.034273301288733


In [14]:
# Crear un DataFrame a partir de la matriz escalada
df_geyser_scaled_df = pd.DataFrame(df_geyser_scaled, columns=['duration', 'waiting'])
df_geyser_scaled_df

,duration,waiting
0,0.098499,0.597123
1,-1.481459,-1.245181
2,-0.135861,0.228663
3,-1.057503,-0.655644
4,0.917443,1.039277
...,...,...
267,0.552298,0.744508
268,-1.174245,-1.834719
269,0.815624,1.407737
270,-1.466537,-1.834719


In [15]:
# Se agrega una columna llamada 'anomaly' al DataFrame
df_geyser_scaled_df['anomaly'] = (densities < density_threshold).astype(int)
df_geyser_scaled_df['anomaly'].value_counts()

anomaly
0    271
1      1
Name: count, dtype: int64

#### Percentile 0.1

In [16]:
# Selección del Threshold
df_geyser_scaled_df.drop(columns='anomaly', inplace=True)
densities = gm.score_samples(df_geyser_scaled_df)
density_threshold = np.percentile(densities, 0.1)
print("Threshold seleccionado:", density_threshold)

Threshold seleccionado: -5.99196693811587


/Users/n.c.rodriguez/anaconda3/envs/nico/lib/python3.10/site-packages/sklearn/base.py:457: UserWarning: X has feature names, but GaussianMixture was fitted without feature names
  warnings.warn(


In [17]:
# Se agrega una columna llamada 'anomaly' al DataFrame
df_geyser_scaled_df['anomaly'] = (densities < density_threshold).astype(int)
df_geyser_scaled_df['anomaly'].value_counts()

anomaly
0    271
1      1
Name: count, dtype: int64

### Búsqueda del mejor Threshold

In [18]:
X = df_geyser_scaled_df.drop(columns='anomaly')
y = df_geyser_scaled_df['anomaly']

In [19]:
from sklearn.base import BaseEstimator

class GaussianAnomalyDetector(BaseEstimator):
    def __init__(self, threshold=1):
        self._threshold = threshold
        self._gm = None
    def fit(self, X, y=None):
        self._gm = GaussianMixture(n_components=2, n_init=10, random_state=42)
        self._gm.fit(X) 
        return self
    
    def predict(self, X, y=None):
        densities = self._gm.score_samples(X)
        y_preds = (densities < self._threshold)
        y_preds[y_preds == False] = 0
        y_preds[y_preds == True] = 1
        return y_preds
    
    def get_params(self, deep=True):
        return {"threshold": self._threshold}

    def set_params(self, **parameters):
        for parameter, value in parameters.items():
            setattr(self, parameter, value)
            return self

In [20]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform

gad = GaussianAnomalyDetector()

param_distribs = {
    # Utiliza 'uniform' para distribuciones continuas. 
    # 'loc' es el inicio del rango y 'scale' es la anchura del rango (el total de valores en el rango).
    'threshold': uniform(loc=0.01, scale=4.99),
}

# Configuración de RandomizedSearchCV (5*3=15 rondas de entrenamiento)
rnd_search = RandomizedSearchCV(gad, param_distributions=param_distribs,
                                n_iter=5, cv=3, scoring='f1')

# Entrenamiento del modelo
rnd_search.fit(X, y)

RandomizedSearchCV(cv=3, estimator=GaussianAnomalyDetector(), n_iter=5,
                   param_distributions={'threshold': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x173092c80>},
                   scoring='f1')

In [21]:
cvres = rnd_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(mean_score, params)

0.007246376811594204 {'threshold': 2.671951213653456}
0.007246376811594204 {'threshold': 1.336381394310952}
0.007246376811594204 {'threshold': 3.0642012927902864}
0.007246376811594204 {'threshold': 0.1580857604371476}
0.007246376811594204 {'threshold': 1.9247566778240641}


La forma mostrada anteriormente es la más estándar a la hora de realizar la búsqueda, pero también es la más ineficiente. Con el método anterior se requiere entrenar el modelo 15 veces (con los parámetro indicados). 

Con la forma que se muestra a continuación solo es necesario entrenar el modelo una única vez.

In [22]:
from sklearn.metrics import precision_score

def select_threshold(list_thds, densities, y):
    best_prec = 0
    best_threshold = 0
    i = 0
    for thd in list_thds:
        i += 1
        print("\rSearching best threshold {0}%".format(
            int((i + 1) / len(list_thds) * 100)), end='')
        preds = (densities < thd)
        preds[preds == False] = 0
        preds[preds == True] = 1
        precision = precision_score(y, preds)
        if precision > best_prec:
            best_prec = precision
            best_threshold = thd
    return (best_prec, best_threshold)

In [23]:
select_threshold(np.arange(-600, -300, 1), densities, y)

Searching best threshold 100%

/Users/n.c.rodriguez/anaconda3/envs/nico/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/n.c.rodriguez/anaconda3/envs/nico/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/n.c.rodriguez/anaconda3/envs/nico/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/n.c.rodriguez/anaconda3/envs/n

(0, 0)